In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, JSON, Table
from sqlalchemy.orm import declarative_base, relationship, sessionmaker

Base = declarative_base()

# Association tables for many-to-many relationships
requirement_function_association = Table(
    'requirement_function', Base.metadata,
    Column('requirement_id', Integer, ForeignKey('requirements.id')),
    Column('function_id', Integer, ForeignKey('functions.id'))
)

function_physical_association = Table(
    'function_physical', Base.metadata,
    Column('function_id', Integer, ForeignKey('functions.id')),
    Column('physical_id', Integer, ForeignKey('physicals.id'))
)

# Self-referential association tables for sub-requirements, sub-functions, and sub-physicals
sub_requirement_association = Table(
    'sub_requirement', Base.metadata,
    Column('parent_id', Integer, ForeignKey('requirements.id')),
    Column('child_id', Integer, ForeignKey('requirements.id'))
)

sub_function_association = Table(
    'sub_function', Base.metadata,
    Column('parent_id', Integer, ForeignKey('functions.id')),
    Column('child_id', Integer, ForeignKey('functions.id'))
)

sub_physical_association = Table(
    'sub_physical', Base.metadata,
    Column('parent_id', Integer, ForeignKey('physicals.id')),
    Column('child_id', Integer, ForeignKey('physicals.id'))
)

# Define the top-level Component model
class Component(Base):
    __tablename__ = 'components'

    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    description = Column(String, nullable=True)

    # One-to-many relationship with requirements
    requirements = relationship("Requirement", back_populates="component")

    def __repr__(self):
        return f"<Component(id={self.id}, name='{self.name}')>"

# Define the Requirement model with JSON data and self-referential sub-requirements
class Requirement(Base):
    __tablename__ = 'requirements'

    id = Column(Integer, primary_key=True)
    component_id = Column(Integer, ForeignKey('components.id'), nullable=True)  # Made nullable for sub-requirements
    data = Column(JSON, nullable=False)  # Store Requirement as JSON

    component = relationship("Component", back_populates="requirements")
    functions = relationship("Function", secondary=requirement_function_association, back_populates="requirements")
    
    # Self-referential relationship for sub-requirements
    sub_requirements = relationship(
        "Requirement", secondary=sub_requirement_association,
        primaryjoin=id == sub_requirement_association.c.parent_id,
        secondaryjoin=id == sub_requirement_association.c.child_id,
        backref="parent_requirements"
    )

    def __repr__(self):
        return f"<Requirement(id={self.id}, data={self.data})>"

# Define the Function model with JSON data and self-referential sub-functions
class Function(Base):
    __tablename__ = 'functions'

    id = Column(Integer, primary_key=True)
    data = Column(JSON, nullable=False)  # Store Function as JSON

    requirements = relationship("Requirement", secondary=requirement_function_association, back_populates="functions")
    physicals = relationship("Physical", secondary=function_physical_association, back_populates="functions")
    
    # Self-referential relationship for sub-functions
    sub_functions = relationship(
        "Function", secondary=sub_function_association,
        primaryjoin=id == sub_function_association.c.parent_id,
        secondaryjoin=id == sub_function_association.c.child_id,
        backref="parent_functions"
    )

    def __repr__(self):
        return f"<Function(id={self.id}, data={self.data})>"

# Define the Physical model with JSON data and self-referential sub-physicals
class Physical(Base):
    __tablename__ = 'physicals'

    id = Column(Integer, primary_key=True)
    data = Column(JSON, nullable=False)  # Store Physical as JSON

    functions = relationship("Function", secondary=function_physical_association, back_populates="physicals")
    
    # Self-referential relationship for sub-physical elements
    sub_physicals = relationship(
        "Physical", secondary=sub_physical_association,
        primaryjoin=id == sub_physical_association.c.parent_id,
        secondaryjoin=id == sub_physical_association.c.child_id,
        backref="parent_physicals"
    )

    def __repr__(self):
        return f"<Physical(id={self.id}, data={self.data})>"

# Database setup
engine = create_engine('sqlite:///rflp_component_json_relations.db')
Base.metadata.create_all(engine)

# Session setup
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
large_json_data2= [
    {
        "component_name": "Electric Scooter",
        "component_description": "Personal electric scooter system",
        "requirements": [
            {
                "description": "Charge time to 80%",
                "spec": "2 hours",
                "sub_requirements": [
                    {"description": "Battery longevity", "spec": "500 cycles"}
                ],
                "functions": [
                    {"description": "Turn on/off", "operation": "switch control"},
                    {"description": "Charging", "operation": "plug-in charging"}
                ],
                "physicals": [
                    {"component": "Battery", "type": "Lithium-ion, 48V"},
                    {"component": "Motor", "type": "Hub motor"}
                ]
            },
            {
                "description": "Power for 40 miles",
                "spec": "10Ah battery",
                "functions": [
                    {"description": "Power output", "operation": "regulated output"}
                ],
                "physicals": [
                    {"component": "Motor controller", "type": "PWM"}
                ]
            }
        ]
    }
]

In [ ]:
def insert_component_data(json_data, session):
    for component_data in json_data:
        # Insert Component
        component = Component(name=component_data['component_name'], description=component_data['component_description'])
        session.add(component)
        session.commit()

        for requirement_data in component_data['requirements']:
            # Insert Requirement
            requirement = Requirement(data={"description": requirement_data['description'], "spec": requirement_data['spec']})
            session.add(requirement)
            session.commit()
            
            # Associate Requirement with Component
            component.requirements.append(requirement)
            session.commit()

            # Insert Sub-Requirements
            for sub_req_data in requirement_data.get('sub_requirements', []):
                sub_requirement = Requirement(data={"description": sub_req_data['description'], "spec": sub_req_data['spec']})
                session.add(sub_requirement)
                session.commit()
                requirement.sub_requirements.append(sub_requirement)
                session.commit()

            # Insert Functions
            for function_data in requirement_data.get('functions', []):
                function = Function(data={"description": function_data['description'], "operation": function_data['operation']})
                session.add(function)
                session.commit()
                requirement.functions.append(function)
                session.commit()

                # Insert Physicals for the Function
                for physical_data in function_data.get('physicals', []):
                    physical = Physical(data={"component": physical_data['component'], "type": physical_data['type']})
                    session.add(physical)
                    session.commit()
                    function.physicals.append(physical)
                    session.commit()

        # Commit the changes for the component and its requirements, functions, etc.
        session.commit()

# Example usage:
insert_component_data(large_json_data2, session)


In [ ]:
# random requirements

def add_requirement(session, component_id, data):
    """
    Adds a new requirement to the database.
    """
    requirement = Requirement(component_id=component_id, data=data)
    session.add(requirement)
    session.commit()
    return requirement


add_requirement(session, component_id=1, data={"title": "Requirement 1", "description": "prahlad"})

In [ ]:
def get_full_hierarchy_for_component(component_id, session):
    """
    Retrieve the full hierarchy for a component from the database, including all requirements, sub-requirements, functions, and physicals.
    """
    def retrieve_sub_requirements(requirement):
        """ Recursively retrieve sub-requirements for a given requirement. """
        sub_requirements_data = []
        for sub_requirement in requirement.sub_requirements:
            sub_requirements_data.append({
                "description": sub_requirement.data.get("description"),
                "spec": sub_requirement.data.get("spec"),
                "sub_requirements": retrieve_sub_requirements(sub_requirement)
            })
        return sub_requirements_data

    def retrieve_sub_functions(function):
        """ Recursively retrieve sub-functions for a given function. """
        sub_functions_data = []
        for sub_function in function.sub_functions:
            sub_functions_data.append({
                "description": sub_function.data.get("description"),
                "operation": sub_function.data.get("operation"),
                "sub_functions": retrieve_sub_functions(sub_function)
            })
        return sub_functions_data

    def retrieve_sub_physicals(physical):
        """ Recursively retrieve sub-physicals for a given physical. """
        sub_physicals_data = []
        for sub_physical in physical.sub_physicals:
            sub_physicals_data.append({
                "component": sub_physical.data.get("component"),
                "type": sub_physical.data.get("type"),
                "sub_physicals": retrieve_sub_physicals(sub_physical)
            })
        return sub_physicals_data

    # Retrieve the component
    component = session.query(Component).filter(Component.id == component_id).first()
    
    if not component:
        raise ValueError(f"Component with id {component_id} not found.")

    # Build the hierarchical JSON structure
    component_data = {
        "component_name": component.name,
        "component_description": component.description,
        "requirements": []
    }

    for requirement in component.requirements:
        requirement_data = {
            "description": requirement.data.get("description"),
            "spec": requirement.data.get("spec"),
            "sub_requirements": retrieve_sub_requirements(requirement),
            "functions": []
        }
        
        for function in requirement.functions:
            function_data = {
                "description": function.data.get("description"),
                "operation": function.data.get("operation"),
                "sub_functions": retrieve_sub_functions(function),
                "physicals": []
            }
            
            for physical in function.physicals:
                physical_data = {
                    "component": physical.data.get("component"),
                    "type": physical.data.get("type"),
                    "sub_physicals": retrieve_sub_physicals(physical)
                }
                function_data["physicals"].append(physical_data)
            
            requirement_data["functions"].append(function_data)
        
        component_data["requirements"].append(requirement_data)

    return [component_data]

# Example usage:
component_id = 1  # Replace with the appropriate component ID you want to retrieve
component_hierarchy = get_full_hierarchy_for_component(component_id, session)

# Printing the full hierarchy as a JSON string
import json
print(json.dumps(component_hierarchy, indent=4))


including the id for each entry

In [ ]:
def get_full_hierarchy_for_component(component_id, session):
    """
    Retrieve the full hierarchy for a component from the database, including all requirements, sub-requirements, functions, and physicals.
    """
    def retrieve_sub_requirements(requirement):
        """ Recursively retrieve sub-requirements for a given requirement. """
        sub_requirements_data = []
        for sub_requirement in requirement.sub_requirements:
            sub_requirements_data.append({
                "id": sub_requirement.id,  # Include the ID
                "description": sub_requirement.data.get("description"),
                "spec": sub_requirement.data.get("spec"),
                "sub_requirements": retrieve_sub_requirements(sub_requirement)
            })
        return sub_requirements_data

    def retrieve_sub_functions(function):
        """ Recursively retrieve sub-functions for a given function. """
        sub_functions_data = []
        for sub_function in function.sub_functions:
            sub_functions_data.append({
                "id": sub_function.id,  # Include the ID
                "description": sub_function.data.get("description"),
                "operation": sub_function.data.get("operation"),
                "sub_functions": retrieve_sub_functions(sub_function)
            })
        return sub_functions_data

    def retrieve_sub_physicals(physical):
        """ Recursively retrieve sub-physicals for a given physical. """
        sub_physicals_data = []
        for sub_physical in physical.sub_physicals:
            sub_physicals_data.append({
                "id": sub_physical.id,  # Include the ID
                "component": sub_physical.data.get("component"),
                "type": sub_physical.data.get("type"),
                "sub_physicals": retrieve_sub_physicals(sub_physical)
            })
        return sub_physicals_data

    # Retrieve the component
    component = session.query(Component).filter(Component.id == component_id).first()
    
    if not component:
        raise ValueError(f"Component with id {component_id} not found.")

    # Build the hierarchical JSON structure
    component_data = {
        "id": component.id,  # Include the ID of the component
        "component_name": component.name,
        "component_description": component.description,
        "requirements": []
    }

    for requirement in component.requirements:
        requirement_data = {
            "id": requirement.id,  # Include the ID of the requirement
            "description": requirement.data.get("description"),
            "spec": requirement.data.get("spec"),
            "sub_requirements": retrieve_sub_requirements(requirement),
            "functions": []
        }
        
        for function in requirement.functions:
            function_data = {
                "id": function.id,  # Include the ID of the function
                "description": function.data.get("description"),
                "operation": function.data.get("operation"),
                "sub_functions": retrieve_sub_functions(function),
                "physicals": []
            }
            
            for physical in function.physicals:
                physical_data = {
                    "id": physical.id,  # Include the ID of the physical
                    "component": physical.data.get("component"),
                    "type": physical.data.get("type"),
                    "sub_physicals": retrieve_sub_physicals(physical)
                }
                function_data["physicals"].append(physical_data)
            
            requirement_data["functions"].append(function_data)
        
        component_data["requirements"].append(requirement_data)

    return [component_data]

# Example usage:
component_id = 1  # Replace with the appropriate component ID you want to retrieve
component_hierarchy = get_full_hierarchy_for_component(component_id, session)

# Printing the full hierarchy as a JSON string
import json
print(json.dumps(component_hierarchy, indent=4))


# New code with UUID

unique id


In [1]:
import uuid
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, JSON, Table
from sqlalchemy.orm import declarative_base, relationship, sessionmaker
from sqlalchemy.dialects.postgresql import UUID

Base = declarative_base()

# Association tables for many-to-many relationships
requirement_function_association = Table(
    'requirement_function', Base.metadata,
    Column('requirement_id', Integer, ForeignKey('requirements.id')),
    Column('function_id', Integer, ForeignKey('functions.id'))
)

function_physical_association = Table(
    'function_physical', Base.metadata,
    Column('function_id', Integer, ForeignKey('functions.id')),
    Column('physical_id', Integer, ForeignKey('physicals.id'))
)

# Self-referential association tables for sub-requirements, sub-functions, and sub-physicals
sub_requirement_association = Table(
    'sub_requirement', Base.metadata,
    Column('parent_id', Integer, ForeignKey('requirements.id')),
    Column('child_id', Integer, ForeignKey('requirements.id'))
)

sub_function_association = Table(
    'sub_function', Base.metadata,
    Column('parent_id', Integer, ForeignKey('functions.id')),
    Column('child_id', Integer, ForeignKey('functions.id'))
)

sub_physical_association = Table(
    'sub_physical', Base.metadata,
    Column('parent_id', Integer, ForeignKey('physicals.id')),
    Column('child_id', Integer, ForeignKey('physicals.id'))
)

# Define the top-level Component model with UUID
class Component(Base):
    __tablename__ = 'components'

    id = Column(Integer, primary_key=True)
    uuid = Column(UUID(as_uuid=True), default=uuid.uuid4, unique=True, nullable=False)  # UUID field
    name = Column(String, nullable=False)
    description = Column(String, nullable=True)

    # One-to-many relationship with requirements
    requirements = relationship("Requirement", back_populates="component")

    def __repr__(self):
        return f"<Component(id={self.id}, uuid={self.uuid}, name='{self.name}')>"

# Define the Requirement model with UUID and JSON data
class Requirement(Base):
    __tablename__ = 'requirements'

    id = Column(Integer, primary_key=True)
    uuid = Column(UUID(as_uuid=True), default=uuid.uuid4, unique=True, nullable=False)  # UUID field
    component_id = Column(Integer, ForeignKey('components.id'), nullable=True)  # Made nullable for sub-requirements
    data = Column(JSON, nullable=False)  # Store Requirement as JSON

    component = relationship("Component", back_populates="requirements")
    functions = relationship("Function", secondary=requirement_function_association, back_populates="requirements")
    
    # Self-referential relationship for sub-requirements
    sub_requirements = relationship(
        "Requirement", secondary=sub_requirement_association,
        primaryjoin=id == sub_requirement_association.c.parent_id,
        secondaryjoin=id == sub_requirement_association.c.child_id,
        backref="parent_requirements"
    )

    def __repr__(self):
        return f"<Requirement(id={self.id}, uuid={self.uuid}, data={self.data})>"

# Define the Function model with UUID and JSON data
class Function(Base):
    __tablename__ = 'functions'

    id = Column(Integer, primary_key=True)
    uuid = Column(UUID(as_uuid=True), default=uuid.uuid4, unique=True, nullable=False)  # UUID field
    data = Column(JSON, nullable=False)  # Store Function as JSON

    requirements = relationship("Requirement", secondary=requirement_function_association, back_populates="functions")
    physicals = relationship("Physical", secondary=function_physical_association, back_populates="functions")
    
    # Self-referential relationship for sub-functions
    sub_functions = relationship(
        "Function", secondary=sub_function_association,
        primaryjoin=id == sub_function_association.c.parent_id,
        secondaryjoin=id == sub_function_association.c.child_id,
        backref="parent_functions"
    )

    def __repr__(self):
        return f"<Function(id={self.id}, uuid={self.uuid}, data={self.data})>"

# Define the Physical model with UUID and JSON data
class Physical(Base):
    __tablename__ = 'physicals'

    id = Column(Integer, primary_key=True)
    uuid = Column(UUID(as_uuid=True), default=uuid.uuid4, unique=True, nullable=False)  # UUID field
    data = Column(JSON, nullable=False)  # Store Physical as JSON

    functions = relationship("Function", secondary=function_physical_association, back_populates="physicals")
    
    # Self-referential relationship for sub-physical elements
    sub_physicals = relationship(
        "Physical", secondary=sub_physical_association,
        primaryjoin=id == sub_physical_association.c.parent_id,
        secondaryjoin=id == sub_physical_association.c.child_id,
        backref="parent_physicals"
    )

    def __repr__(self):
        return f"<Physical(id={self.id}, uuid={self.uuid}, data={self.data})>"

# Database setup
engine = create_engine('sqlite:///db_model_3.db')
Base.metadata.create_all(engine)

# Session setup
Session = sessionmaker(bind=engine)
session = Session()


In [2]:
def get_full_hierarchy_for_component(component_id, session):
    """
    Retrieve the full hierarchy for a component from the database, including all requirements, sub-requirements, functions, and physicals.
    """
    def retrieve_sub_requirements(requirement):
        """ Recursively retrieve sub-requirements for a given requirement. """
        sub_requirements_data = []
        for sub_requirement in requirement.sub_requirements:
            sub_requirements_data.append({
                "id": sub_requirement.id,  # Include the ID
                "uuid": str(sub_requirement.uuid),  # Include the UUID
                "description": sub_requirement.data.get("description"),
                "spec": sub_requirement.data.get("spec"),
                "sub_requirements": retrieve_sub_requirements(sub_requirement)
            })
        return sub_requirements_data

    def retrieve_sub_functions(function):
        """ Recursively retrieve sub-functions for a given function. """
        sub_functions_data = []
        for sub_function in function.sub_functions:
            sub_functions_data.append({
                "id": sub_function.id,  # Include the ID
                "uuid": str(sub_function.uuid),  # Include the UUID
                "description": sub_function.data.get("description"),
                "operation": sub_function.data.get("operation"),
                "sub_functions": retrieve_sub_functions(sub_function)
            })
        return sub_functions_data

    def retrieve_sub_physicals(physical):
        """ Recursively retrieve sub-physicals for a given physical. """
        sub_physicals_data = []
        for sub_physical in physical.sub_physicals:
            sub_physicals_data.append({
                "id": sub_physical.id,  # Include the ID
                "uuid": str(sub_physical.uuid),  # Include the UUID
                "component": sub_physical.data.get("component"),
                "type": sub_physical.data.get("type"),
                "sub_physicals": retrieve_sub_physicals(sub_physical)
            })
        return sub_physicals_data

    # Retrieve the component
    component = session.query(Component).filter(Component.id == component_id).first()
    
    if not component:
        raise ValueError(f"Component with id {component_id} not found.")

    # Build the hierarchical JSON structure
    component_data = {
        "id": component.id,  # Include the ID of the component
        "uuid": str(component.uuid),  # Include the UUID of the component
        "component_name": component.name,
        "component_description": component.description,
        "requirements": []
    }

    for requirement in component.requirements:
        requirement_data = {
            "id": requirement.id,  # Include the ID of the requirement
            "uuid": str(requirement.uuid),  # Include the UUID of the requirement
            "description": requirement.data.get("description"),
            "spec": requirement.data.get("spec"),
            "sub_requirements": retrieve_sub_requirements(requirement),
            "functions": []
        }
        
        for function in requirement.functions:
            function_data = {
                "id": function.id,  # Include the ID of the function
                "uuid": str(function.uuid),  # Include the UUID of the function
                "description": function.data.get("description"),
                "operation": function.data.get("operation"),
                "sub_functions": retrieve_sub_functions(function),
                "physicals": []
            }
            
            for physical in function.physicals:
                physical_data = {
                    "id": physical.id,  # Include the ID of the physical
                    "uuid": str(physical.uuid),  # Include the UUID of the physical
                    "component": physical.data.get("component"),
                    "type": physical.data.get("type"),
                    "sub_physicals": retrieve_sub_physicals(physical)
                }
                function_data["physicals"].append(physical_data)
            
            requirement_data["functions"].append(function_data)
        
        component_data["requirements"].append(requirement_data)
    
    return component_data


In [3]:
large_json_data2= [
    {
        "component_name": "Electric Scooter",
        "component_description": "Personal electric scooter system",
        "requirements": [
            {
                "description": "Charge time to 80%",
                "spec": "2 hours",
                "sub_requirements": [
                    {"description": "Battery longevity", "spec": "500 cycles"}
                ],
                "functions": [
                    {"description": "Turn on/off", "operation": "switch control"},
                    {"description": "Charging", "operation": "plug-in charging"}
                ],
                "physicals": [
                    {"component": "Battery", "type": "Lithium-ion, 48V"},
                    {"component": "Motor", "type": "Hub motor"}
                ]
            },
            {
                "description": "Power for 40 miles",
                "spec": "10Ah battery",
                "functions": [
                    {"description": "Power output", "operation": "regulated output"}
                ],
                "physicals": [
                    {"component": "Motor controller", "type": "PWM"}
                ]
            }
        ]
    }
]

In [4]:
def insert_component_data(json_data, session):
    try:
        for component_data in json_data:
            # Insert Component
            component = Component(
                name=component_data['component_name'], 
                description=component_data['component_description']
            )
            session.add(component)

            for requirement_data in component_data['requirements']:
                # Insert Requirement
                requirement = Requirement(
                    data={
                        "description": requirement_data['description'], 
                        "spec": requirement_data['spec']
                    }
                )
                component.requirements.append(requirement)

                # Insert Sub-Requirements
                for sub_req_data in requirement_data.get('sub_requirements', []):
                    sub_requirement = Requirement(
                        data={
                            "description": sub_req_data['description'], 
                            "spec": sub_req_data['spec']
                        }
                    )
                    requirement.sub_requirements.append(sub_requirement)

                # Insert Functions
                for function_data in requirement_data.get('functions', []):
                    function = Function(
                        data={
                            "description": function_data['description'], 
                            "operation": function_data['operation']
                        }
                    )
                    requirement.functions.append(function)

                    # Insert Physicals for the Function
                    for physical_data in function_data.get('physicals', []):
                        physical = Physical(
                            data={
                                "component": physical_data['component'], 
                                "type": physical_data['type']
                            }
                        )
                        function.physicals.append(physical)

        # Commit all changes in a single transaction
        session.commit()

    except Exception as e:
        # Rollback in case of any error
        session.rollback()
        print(f"Error occurred: {e}")
        raise
    
    
# Example usage:
insert_component_data(large_json_data2, session)


In [8]:
# First, we need to create some sample data and add it to the database.

from sqlalchemy.orm import sessionmaker
import uuid

# Creating a session
Session = sessionmaker(bind=engine)
session = Session()

# Creating a new component
component = Component(name="Example Component", description="This is a test component.")
session.add(component)
session.commit()  # Commit to save the component

# Creating requirements for the component
requirement = Requirement(data={"description": "Test Requirement", "spec": "Spec 1"})
requirement.component = component  # Link the requirement to the component
session.add(requirement)
session.commit()

# Creating a function for the requirement
function = Function(data={"description": "Test Function", "operation": "Operation 1"})
requirement.functions.append(function)  # Associate function with the requirement
session.add(function)
session.commit()

# Creating a physical element for the function
physical = Physical(data={"component": "Test Component", "type": "Type 1"})
function.physicals.append(physical)  # Associate physical with the function
session.add(physical)
session.commit()

# Commit and close session after inserting data
session.commit()


In [10]:
# Fetch the hierarchy for the component with the given ID (assuming component.id = 1 for this example)
component_id = 2

try:
    hierarchy = get_full_hierarchy_for_component(component_id, session)
    print("Full Hierarchy for Component:")
    print(hierarchy)
except ValueError as e:
    print(str(e))


import json
print(json.dumps(hierarchy, indent=4))

Full Hierarchy for Component:
{'id': 2, 'uuid': '10402c64-f1b6-4cec-83f8-cc5c46ecab54', 'component_name': 'Example Component', 'component_description': 'This is a test component.', 'requirements': [{'id': 4, 'uuid': '816b516f-41d0-4d10-8371-30a09698cf48', 'description': 'Test Requirement', 'spec': 'Spec 1', 'sub_requirements': [], 'functions': [{'id': 4, 'uuid': '8b263b7d-e668-47ee-b200-115667e9310f', 'description': 'Test Function', 'operation': 'Operation 1', 'sub_functions': [], 'physicals': [{'id': 1, 'uuid': 'd9c3326c-f03b-41f0-951b-96105d961c93', 'component': 'Test Component', 'type': 'Type 1', 'sub_physicals': []}]}]}]}
{
    "id": 2,
    "uuid": "10402c64-f1b6-4cec-83f8-cc5c46ecab54",
    "component_name": "Example Component",
    "component_description": "This is a test component.",
    "requirements": [
        {
            "id": 4,
            "uuid": "816b516f-41d0-4d10-8371-30a09698cf48",
            "description": "Test Requirement",
            "spec": "Spec 1",
        

# Versioned

In [6]:
from sqlalchemy.orm import sessionmaker
import uuid

# Creating a session
Session = sessionmaker(bind=engine)
session = Session()

# Creating a new component with initial version
component = Component(
    name="Example Component", 
    description="This is a test component.", 
    version=1  # Initial version
)
session.add(component)
session.commit()  # Commit to save the component

# Creating a requirement for the component with initial version
requirement = Requirement(
    data={"description": "Test Requirement", "spec": "Spec 1"},
    version=1  # Initial version
)
requirement.component = component  # Link the requirement to the component
session.add(requirement)
session.commit()

# Creating a function for the requirement with initial version
function = Function(
    data={"description": "Test Function", "operation": "Operation 1"},
    version=1  # Initial version
)
requirement.functions.append(function)  # Associate function with the requirement
session.add(function)
session.commit()

# Creating a physical element for the function with initial version
physical = Physical(
    data={"component": "Test Component", "type": "Type 1"},
    version=1  # Initial version
)
function.physicals.append(physical)  # Associate physical with the function
session.add(physical)
session.commit()

# Commit and close session after inserting data
session.commit()
session.close()


AttributeError: 'Requirement' object has no attribute 'functions'